In [ ]:
%matplotlib inline
import MySQLdb as mdb
import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
sns.set()

from datetime import datetime
import re

matplotlib.style.use('ggplot')
#pd.set_option('display.notebook_repr_html', True)

# Helper Functions 

### Fetch Data From MySQL

In [ ]:
def fetch(cols,table,where=None,group_by=None,order_by=None,Desc=False,limit=None):
    """
    Purpose: Makes the MySQL quesries more efficitent and faster.
    
    Note: All inputs have to be in STRING format except the 'limit' which is an int
    """
    
    # Opening a Connection to donorsChoose Database
    con = mdb.connect('localhost', 'idx', 'donorsChoose', 'donors');
    cur = con.cursor()
    
    # Creating a query
    query = 'SELECT ' + cols + ' FROM ' + table + ' '
    if where: query += ' WHERE ' + where + ' '
    if group_by: query += 'GROUP BY ' + group_by + ' '
    if order_by: query += 'ORDER BY ' + order_by + ' '
    if Desc: query += 'DESC '
    if limit: query += 'LIMIT ' + str(limit)
    
    # fetching the data and turn it into a pandas dataframe
    cur.execute(query)
    output = np.array(cur.fetchall())
    colm = [f[0] for f in cur.description]
    output = pd.DataFrame(output)
    output.columns = colm

    # Closing the connection to the database
    con.close()
    
    return output

### Statistical Analysis Functions 

In [ ]:
plot_proportions('primary_focus_subject',figSize=(14,10))

Grouping is not done solely based on similarity of the subjects but also funding status distribution. For example, It seems that math & literacy have a similar popularity level in LA!

Reducing the levels to:


- music and art ('Visual Arts','Music','Performing Arts')
- community related ('Early Development','Environmental Science','Character Education','Community Service','Applied Sciences')
- health and sports ('Health & Life Science','Health & Wellness','Gym & Fitness', 'Sports','Nutrition','Special Needs')
- literacy and math ('Literature & Writing','Literacy','Foreign Languages','ESL','Mathematics')
- history and civics ('Social Sciences','History & Geography','Civics & Government','Economics')
- Other ('Other','Extracurricular','College & Career Prep','Parent Involvement','')

In [ ]:
primary_subject_mapping = {'Visual Arts': 'music_art', 'Music': 'music_art', 'Performing Arts': 'music_art',
                          'Early Development': 'community_related', 'Environmental Science': 'community_related',
                          'Character Education': 'community_related', 'Community Service': 'community_related',
                           'Applied Sciences': 'community_related', 'Health & Life Science': 'health' , 
                           'Health & Wellness': 'health', 'Gym & Fitness': 'health', 'Sports': 'health',
                           'Nutrition': 'health', 'Literature & Writing': 'literacy_math', 'Literacy': 'literacy_math',
                           'Foreign Languages': 'literacy_math', 'ESL': 'literacy_math' ,'Mathematics': 'literacy_math',
                           'Social Sciences': 'history_civics','History & Geography': 'history_civics',
                           'Civics & Government': 'history_civics', 'Economics': 'history_civics', 'Other': 'other',
                           'Extracurricular': 'other', 'College & Career Prep': 'other', 'Special Needs': 'health',
                           'Parent Involvement': 'other','': 'other'}

projects_la['primary_focus_subject_grouped'] = projects_la.primary_focus_subject.map(primary_subject_mapping).astype(object)

# Have to apply the same grouping to the secondary subject
projects_la['secondary_focus_subject_grouped'] = projects_la.secondary_focus_subject.map(primary_subject_mapping).astype(object)

In [ ]:
plot_proportions('primary_focus_subject_grouped',figSize=(7,4))

In [ ]:
print 'chi2 p-value before grouping:', chi2_test('primary_focus_subject')
print 'chi2 p-value after grouping:', chi2_test('primary_focus_subject_grouped')

Fantastic! After grouping, p-value is much more significant that what we got for primary_focus_area even that primary_focus_area has more levels.

### Poverty Level

It is interesting that schools with lower poverty level are more likely to receive the fund! But their fraction is much lower. So in general more money goes to high poverty schools.

Let's make it only two levels: Low & High Poverty.

In [ ]:
plot_proportions('poverty_level',figSize=(7,4))

In [ ]:
projects_la.poverty_level.value_counts()

In [ ]:
poverty_level_mapping = {'highest poverty': 'high', 'high poverty': 'high',
                         'moderate poverty': 'low', 'low poverty': 'low'}
projects_la['poverty_level_grouped'] = projects_la.poverty_level.map(poverty_level_mapping).astype(object)

### Teacher's Gender

School zipcode, city, and district are clearly correlated. Out this three, I will include only the school_zipcode in building predictive models. It's more intuitive and also $\chi^2$ test confirmed that it has the highest degree of variation among the three.

## Grouping The Levels to Prevent Overfitting

### Resource Type

In [ ]:
projects_la.resource_type.value_counts()

In [ ]:
plot_proportions('resource_type',figSize=(7,4))

Reducing the levels to:

- trips and visitor 
- book and supplies
- Technology and other.

In [ ]:
resource_type_mapping = {'Trips': 'Trips_Visitor', 'Visitors': 'Trips_Visitor',
                         'Books': 'Books_Supplies', 'Supplies': 'Books_Supplies',
                         'Other': 'Technology_other', 'Technology': 'Technology_other', '':'Technology_other'}
projects_la['resource_type_grouped'] = projects_la.resource_type.map(resource_type_mapping).astype(object)

In [ ]:
plot_proportions('resource_type_grouped',figSize=(7,3))

In [ ]:
print 'chi2 p-value before grouping:', chi2_test('resource_type')
print 'chi2 p-value after grouping:', chi2_test('resource_type_grouped')

Not bad! $10^{-132}$ is still pretty good! haha! Handling three levels is for sure easier.

### Project Focus Subject & Area

DonorsChoose 'Focus Area' is one way of grouping the 'Focus Subjects'. This grouping is based on context similarity of different subjects. For example, three 'Music & The Arts' different subjects fit into this docus area. See below for more details:

In [ ]:
print '(numofRows, numofColms) =' , projects_la.shape

In [ ]:
projects_la.head()

To make our life easier, let's create a numerical binary column for funding status

In [ ]:
# Have a column of success and failures as 1 & 0 for the ease of calculation
projects_la['funding_status_binary'] = projects_la.funding_status.map( {'completed': 1, 'expired': 0}).astype(int)

### Dropping Unrelevant Columns

Information about the state/county and 'teacher_ny_teaching_fellow' are clearly redundant; these are projecy LA county. Also, it turned out there is no New Leaders New Schools (nlns) in LA. 

In [ ]:
projects_la.drop(['school_state','school_county','teacher_ny_teaching_fellow','school_nlns'], axis=1, inplace=True)
print '(numofRows, numofColms) =' , projects_la.shape

# Correcting for MySQL dtype Assignments

The fetched data through MySQL are all labeled as object (categorical) data type. Need to correct that for numerical and timestamp data.

### Correting Numerical Columns dtype

In [ ]:
numerical_columns = ['vendor_shipping_charges', 'sales_tax', 'payment_processing_charges', \
                     'fulfillment_labor_materials','total_price_excluding_optional_support',\
                     'total_price_including_optional_support', 'students_reached','total_donations','num_donors']

projects_la[numerical_columns] = projects_la[numerical_columns].astype(float)

### Correting Timestamp Columns dtype

In [ ]:
plot_proportions('teacher_prefix',figSize=(7,4))

Will group them to male and female. What about the doctors? There are only three projects and it's written by the same person. I am going to assume it's a guy.

In [ ]:
projects_la[projects_la.teacher_prefix == 'Dr.']['_teacher_acctid']

In [ ]:
teacher_gender_mapping = {'Dr.': 'male', 'Mr.': 'male',
                         'Ms.': 'female', 'Mrs.': 'female'}
projects_la['teacher_gender'] = projects_la.teacher_prefix.map(teacher_gender_mapping).astype(object)

In [ ]:
plot_proportions('teacher_gender',figSize=(7,4))

# Analyzing Numerical Variables

In [ ]:
numerical_variables = projects_la[['vendor_shipping_charges', 'sales_tax','payment_processing_charges', 
                                    'fulfillment_labor_materials','total_price_excluding_optional_support',
                                    'total_price_including_optional_support', 'students_reached',
                                    'total_donations','num_donors']]

### Comparing the Medians of Funded and Not Funded Projects Using Bootstrap 

In [ ]:
projects_la.groupby('month_posted').funding_status_binary.mean().plot('bar')

Wow! A big difference. Let's group them into first and second semester. 

In [ ]:
projects_la['semester_posted'] = 0

for index,row in projects_la.iterrows():
    if row['month_posted'] < 7:
        projects_la.ix[index,'semester_posted'] = '2nd'
    elif row['month_posted'] >= 7:
        projects_la.ix[index,'semester_posted'] = '1st'

In [ ]:
projects_la.semester_posted.value_counts()

In [ ]:
plot_proportions('semester_posted',figSize=(7,3))

Very good!

### Success Rate vs. Year Posted

In [ ]:
projects_la.groupby('year_posted').funding_status_binary.mean().plot('bar')

Interesting! 2008 collapse clearly had an impact. Anyway, the year posted in not going to be part of my modeling.

# More Feature Engineering

Possibly, teachers who have posted more projects in the past have a higher chance. 

## Number of Teacher's Previous Projects 

In [ ]:
def chi2_test(colName):
    '''
    Purpose: Performs Chi2 independet test on a categorical variable column
    and return the associated p-value.
    
    Note: This function is specifically designed for LA projects and the 
    independent test perfomed based on the funding status (funded or not)
    '''
    
    # Create a multiIndex_table table based on the variable and funding status
    multiIndex_table = projects_la.groupby([colName,'funding_status'])['funding_status'].count()
               
    # Turning funding status into two columns (completed/expired)
    unstacked = multiIndex_table.unstack('funding_status')

    # Making sure that there are at least 10 observation per case
    unstacked = unstacked[unstacked.completed >= 10]
    unstacked = unstacked[unstacked.expired >= 10]
    
    Chi2, p_value, dof, expected = scipy.stats.chi2_contingency(unstacked)
    
    return p_value

In [ ]:
def bootstrap_median(colName,nofSimulation=1000,nofSample=1000,plot_hist=False):
    """
    Purpose: Performs two groups independent tests and compares their median
    
    Note: This function is specifically designed for LA projects and the 
    independent test perfomed based on the funding status (funded or not)
    """
    
    median_diff = []
    
    Funded    = projects_la[projects_la.funding_status=='completed']
    notFunded = projects_la[projects_la.funding_status=='expired']

    # Bootstraping: finding the median differences and severl (nofSimulation) resample of
    # funded and not funded projects. 
    for i in xrange(nofSimulation):
        median1 = Funded[colName].iloc[np.random.randint(0, len(Funded[colName]), size=nofSample)].median()
        median2 = notFunded[colName].iloc[np.random.randint(0, len(notFunded[colName]), size=nofSample)].median()
        median_diff.append(median1 - median2)

    # The number of simulations and resample size will be more than
    # couple of thousands --> use normal z test (CLT)
    mean = np.mean(median_diff)
    SD = np.std(median_diff)
    z = mean/SD
    z = min(-z,z)  # Two sided test --> use the lower quantile
    pValue = 2*stats.norm.cdf(z)
    percLow, percHigh = np.percentile(median_diff,[2.5,97.5])


    print 'The mean of median difference of {0} is {1}, the confidence interval=({2},{3}), and  p-value={4}.'.\
            format(colName,round(mean,2),round(percLow,2),round(percHigh,2),round(pValue,2))
    print
    
    if plot_hist:
        plt.hist(median_diff)   # Optional 

In [ ]:
def sampleProportions_chanceOF_success(colName):
    """
    Purpose: Determine wether or not a binary variable contributes to the success probability
    
    Example: Is being a charter school on not correlated to the chance of getting funded.
    
    Note: This function is specifically designed for LA projects and the 
    independent test perfomed based on the funding status (funded or not)
    """
    
    # prevalence of the binary varible: what is the prevalence of the charter school?
    prevalence = float(sum(projects_la[colName] == '1'))

    # Conditional Probability: If it's a charter school, what is the chance of getting funded?
    success = sum(projects_la[projects_la[colName] == '1']['funding_status'] == 'completed')
    
    # Overal success rate: independent of wether or not it's a charter school
    overal_success = sum(projects_la.funding_status == 'completed')/float(projects_la.shape[0])
    
    # funded given it's a charter school
    p = success/prevalence
    sigma = np.sqrt(p*(1-p)/prevalence)
    percLow = p - 1.96*sigma
    percHigh = p + 1.96*sigma
    
    # Confident interval of success rate compare to the overal success
    return (percLow - overal_success, percHigh - overal_success)

### Plotting 

In [ ]:
def plot_proportions(colName,figSize = (14,19)):
    """
    Purpose: Plots stacked barplots for categorical variable
    
    Note: This function is specifically designed for LA projects and the 
    independent test perfomed based on the funding status (funded or not)
    """
    
    # Create a multiIndex_table table based on the variable and funding status
    multiIndex_table = projects_la.groupby([colName,'funding_status'])['funding_status'].count()
    
    # Turning one of the indices into a column
    multiIndex_table = multiIndex_table.unstack('funding_status')
    
    # Normalizing each row
    multiIndex_table = multiIndex_table.div(multiIndex_table.sum(axis=1), axis=0)
    
    # Making it more readable
    font = {'weight' : 'bold', 'size': 13}
    matplotlib.rc('font', **font)

    ax = multiIndex_table.sort('completed').plot(kind='barh',stacked=True,figsize=figSize)
    ax.set_title('Percentage Funded vs. ' + colName)

# Fetching LA Projects

DonorsChoose data is already parsed and stored on the local machine and can be access via a MySQL connection. For now, I am going to work on projects data only. I will look at the donations and essays separately.

Let's retrieve all the projects that are done in LA county.

In [ ]:
projects_la = fetch('*','projects',where="school_county='Los Angeles'")

In [ ]:
list(projects_la.primary_focus_area.unique())

In [ ]:
list(projects_la.primary_focus_subject.unique())

The goal is to regroup the subjects heuristically to gain more variation (information).

In [ ]:
sns.boxplot(np.log(projects_la.previousProposal_Teacher),groupby=projects_la.funding_status)

Okay, so there is a difference.

## Number of School Previous Projects 

In [ ]:
projects_la['previousProposal_School'] = 0

# Create a dictionary for the number of previouse proposals by a teacher
# All initial values are zero
school_dict = projects_la.set_index('_schoolid')['previousProposal_School'].to_dict()
for each in projects_la._schoolid:
    school_dict[each] += 1

# Create a list of all values --> much faster than updating each row
previousProposal = []
for i in xrange(len(projects_la._schoolid)):
    previousProposal.append(school_dict[projects_la._schoolid[i]])

# Finally updating the column
projects_la['previousProposal_School'] = previousProposal

In [ ]:
bootstrap_median('previousProposal_School',2000,2000)

In [ ]:
sns.boxplot(projects_la.previousProposal_School,groupby=projects_la.funding_status)

## Number of Donors  & Total Donations  per Zipcode

Does it matter how many donors and how much have contrbuted to the projects in LA per zipcode.

In [ ]:
num_donors_per_zip = projects_la.groupby('school_zip').num_donors.sum()
num_donors_per_zip_dict = num_donors_per_zip.to_dict()
projects_la['donor_per_zip'] = projects_la.school_zip.map(lambda x: num_donors_per_zip_dict[x])

donations_per_zip = projects_la.groupby('school_zip').total_donations.sum()
donations_per_zip_dict = donations_per_zip.to_dict()
projects_la['donation_per_zip'] = projects_la.school_zip.map(lambda x: donations_per_zip_dict[x])

projects_la.head()

In [ ]:
bootstrap_median('donation_per_zip',nofSimulation=2000,nofSample=2000)
bootstrap_median('donor_per_zip',nofSimulation=2000,nofSample=2000)

Nope! No dependence!

# Writing The Potentially Important Variable

In [ ]:
for each in numerical_variables:
    bootstrap_median(each,2000,2000)

Not surprisingly, number of donors and total donations are strong indicators of the success. But it's interesting to see how the amount of money teachers ask for is so effective. Sales tax, shipping, and processing charges are highly correlated with the total price. And it seems that fulfillment fee is independent of funding status. 

In [ ]:
timestamp_columns = ['date_posted', 'date_completed','date_thank_you_packet_mailed','date_expiration']

for column in timestamp_columns:
    projects_la[column] = pd.to_datetime(projects_la[column])

In [ ]:
projects_la.dtypes.value_counts()

Okay, everything looks good. Let's see how LA is doing.

# LA in the Last 10 Years

What fraction of the projects are funded? How much money is raised? What about not funded projects?

In [ ]:
sns.set_style("darkgrid",{"xtick.major.size": 1, "ytick.major.size": 1,'axes.labelsize': 15,\
                          'axes.titlesize': 32, 'font.size': 32, 'legend.fontsize': 32.0})

g = sns.boxplot(np.log10(projects_la.total_price_including_optional_support), groupby=projects_la.funding_status,\
            names=['Funded','Not Funded'], fliersize=1, linewidth=5, widths=0.8, saturation=0.7, \
            label='log10(Total Price)')
g.set(ylim=(1.8, 3.8))
g.set_title('Low Cost Projects Are More Likely To Get Funded')
sns.axlabel('Funding Status','log( Total Price )')

# Analyzing Timestamp Variables

Let's see in there is a correlation between the success rate and the project posting dates in terms of year, month, and weekdays.

In [ ]:
projects_la['weekday_posted'] = projects_la['date_posted'].apply(lambda x: x.weekday())
projects_la['month_posted'] = projects_la['date_posted'].apply(lambda x: x.month)
projects_la['year_posted'] = projects_la['date_posted'].apply(lambda x: x.year)

### Success Rate vs. Weekday Posted 

In [ ]:
projects_la.groupby('weekday_posted').funding_status_binary.mean().plot('bar')

Doesn't seem like it!

### Success Rate vs. Month Posted 

In [ ]:
enhanced_chance = pd.DataFrame([sampleProportions_chanceOF_success(x) for x in binary_variables]).T
enhanced_chance.columns = binary_variables.columns
enhanced_chance.index = [['2.5%_percentile_Increased_Chance_of_Success',\
                          '97.5%_percentile_Increased_Chance_of_Success']]
enhanced_chance.T.sort(['2.5%_percentile_Increased_Chance_of_Success',\
                          '97.5%_percentile_Increased_Chance_of_Success'],ascending=False).T

Here is the apparent order of predictive values of the binary variables:

1. eligible_double_your_impact_match (half of the projects is covered by a sponsor)
2. teacher_teach_for_america
3. eligible_almost_home_match (the last $100 of the price is covered by a sponsor)
4. school_charter_ready_promise
5. school_charter
6. school_year_round
7. school_kipp
8. school_magnet

# Analyzing Categorical Variables with More than Two Levels

In [ ]:
categorical_variables = projects_la[['school_city','school_zip','school_metro','school_district','teacher_prefix',
                                     'primary_focus_subject','primary_focus_area','secondary_focus_subject',
                                     'secondary_focus_area', 'resource_type','poverty_level', 'grade_level']]

In [ ]:
independent_test_Pvalue = pd.DataFrame([chi2_test(x) for x in categorical_variables]).T
independent_test_Pvalue.columns = categorical_variables.columns
independent_test_Pvalue.index = ['independent_test_Pvalue']
independent_test_Pvalue.T.sort(['independent_test_Pvalue'],ascending=True).T

Here is the apparent order of predictive values of the categorical variables based on the $\chi^2$ independent test p-values:

1. resource_type
2. school_zip
3. school_city
4. primary_focus_subject
5. school_district
6. primary_focus_area
7. secondary_focus_subject
8. poverty_level
9. secondary_focus_area	
10. grade_level
11. school_metro
12. teacher_prefix

In [ ]:
projects_la['previousProposal_Teacher'] = 0

# Create a dictionary for the number of previouse proposals by a teacher
# All initial values are zero
teach_dict = projects_la.set_index('_teacher_acctid')['previousProposal_Teacher'].to_dict()
for each in projects_la._teacher_acctid:
    teach_dict[each] += 1

# Create a list of all values --> much faster than updating each row
previousProposal = []
for i in xrange(len(projects_la._teacher_acctid)):
    previousProposal.append(teach_dict[projects_la._teacher_acctid[i]])

# Finally updating the column
projects_la['previousProposal_Teacher'] = previousProposal

In [ ]:
bootstrap_median('previousProposal_Teacher',2000,2000)

In [ ]:
projects_la.groupby('funding_status').total_price_including_optional_support.sum()

In [ ]:
# make a square figure and axes
plt.rcParams['font.size'] = 15.0
plt.figure(1, figsize=(6,6))
ax = plt.axes([0.1, 0.1, 0.8, 0.8])

# The slices will be ordered and plotted counter-clockwise.
labels = ['Funded ($26M)', 'Not Funded ($13M)']
fracs = [24640797.09, 12833492.76]
explode=(0, 0.05)
colors=( 'c', 'burlywood')

patches, texts, autotext = plt.pie(fracs, explode=explode, labels=labels ,colors=colors,
                autopct='%1.1f%%', shadow=True, startangle=90)
                # The default startangle is 0, which would start
                # the Frogs slice on the x-axis.  With startangle=90,
                # everything is rotated counter-clockwise by 90 degrees,
                # so the plotting starts on the positive y-axis.

# Changing the labels font size
texts[0].set_fontsize(20)
texts[1].set_fontsize(20)

plt.title('$13M Worth Of Projects Did Not Receive Funding', bbox={'facecolor':'0.9', 'pad':7})

plt.show()

In [ ]:
projects_la.groupby('funding_status').students_reached.sum()

In [ ]:
# make a square figure and axes
plt.rcParams['font.size'] = 15.0
plt.figure(1, figsize=(6,6))
ax = plt.axes([0.1, 0.1, 0.8, 0.8])

# The slices will be ordered and plotted counter-clockwise.
labels = 'Funded (2.8M)', 'Not Funded (2.2M)'
fracs = [2895658 , 2202529 ]
explode=(0, 0.05)
colors=( 'c', 'burlywood')

patches, texts, autotext = plt.pie(fracs, explode=explode, labels=labels ,colors=colors,
                autopct='%1.1f%%', shadow=True, startangle=90)
                # The default startangle is 0, which would start
                # the Frogs slice on the x-axis.  With startangle=90,
                # everything is rotated counter-clockwise by 90 degrees,
                # so the plotting starts on the positive y-axis.

# Changing the labels font size
texts[0].set_fontsize(20)
texts[1].set_fontsize(20)

plt.title('2.2M Students Went Without Beneficiary Support', bbox={'facecolor':'0.9', 'pad':7})

plt.show()

# Analyzing Categorical Variables with Two Levels

They are several binary variables but only two classes: type of school, type of teacher, and different types of promotions. Let's see what's the enhanced chance of success for each of these variables.

In [ ]:
binary_variables = projects_la[['school_charter','school_magnet','school_year_round',\
                            'school_kipp','school_charter_ready_promise','teacher_teach_for_america',\
                            'eligible_double_your_impact_match','eligible_almost_home_match']]